In [1]:
import sys
sys.path.append("../")

import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
config = dotenv_values(env_path)

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
from core.utils.ql_loader import ql
from datetime import datetime
from typing import List

from core.IRSwaps import IRSwaps, IRSwapQuery, IRSwapValue, IRSwapStructure, IRSwapStructureFunctionMap, IRSwapValueFunctionMap
from core.IRSwaptions import IRSwaptions, IRSwaptionQuery, IRSwaptionValue, IRSwaptionStructure, IRSwaptionStructureFunctionMap, IRSwaptionValueFunctionMap
from core.DataFetching.FixingsFetcher import FixingsFetcher
from core.utils.ql_utils import ql_date_to_datetime, datetime_to_ql_date

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib.dates as mdates
plt.style.use('ggplot')
params = {'legend.fontsize': 'x-large',
        'figure.figsize': (12, 8),
        'axes.labelsize': 'x-large',
        'axes.titlesize':'x-large',
        'xtick.labelsize':'x-large',
        'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

import nest_asyncio
nest_asyncio.apply()

In [3]:
curve = "USD-SOFR-1D"
swaps_data_source = rf"CSV_{os.path.join(os.getcwd(), "..", "data", "usd_ois.csv")}"

ff = FixingsFetcher(fred_api_key=config["FRED_API_KEY"]) 
fixings = ff.get_fixings("USD-SOFR-1D")

usd_ois = IRSwaps(
    curve=curve,
    data_source=swaps_data_source,
    fixings=fixings,
    ql_interpolation_algo="log_linear",
    # pre_fetch_curves=True,
    error_verbose=True,
    max_njobs=-1,
)

In [4]:
curves = usd_ois.fetch_ql_irswap_curves(start_date=datetime(2025, 4, 1), end_date=datetime(2025, 5, 1))
curves

{Timestamp('2025-04-01 21:00:00+0000', tz='UTC'): <QuantLib.QuantLib.DiscountCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedDiscountCurve< LogLinear > > *' at 0x7f4b58533f00> >,
 Timestamp('2025-04-02 21:00:00+0000', tz='UTC'): <QuantLib.QuantLib.DiscountCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedDiscountCurve< LogLinear > > *' at 0x7f4b57f97750> >,
 Timestamp('2025-04-03 21:00:00+0000', tz='UTC'): <QuantLib.QuantLib.DiscountCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedDiscountCurve< LogLinear > > *' at 0x7f4b57f97540> >,
 Timestamp('2025-04-04 21:00:00+0000', tz='UTC'): <QuantLib.QuantLib.DiscountCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedDiscountCurve< LogLinear > > *' at 0x7f4b57f97630> >,
 Timestamp('2025-04-07 21:00:00+0000', tz='UTC'): <QuantLib.QuantLib.DiscountCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedDiscountCurve< LogLinear > > *' at 0x7f4b57f97270> >,
 Timestamp

In [6]:
qs = []
for f in usd_ois._DEFAULT_FWD_TENORS:
    for u in usd_ois._DEFAULT_UNDERLYING_TENORS:
        qs.append(IRSwapQuery(tenor=f"{f}x{u}" if f != "0D" else u, structure=IRSwapStructure.OUTRIGHT))

In [4]:
ts_df = usd_ois.irswaps_timeseries_builder(
    start_date=datetime(2020, 1, 1),
    end_date=datetime(2025, 5, 10),
    queries=[
        IRSwapQuery(tenor="10Yx10Y", structure=IRSwapStructure.OUTRIGHT),
        IRSwapQuery(tenor="1Yx10Y", structure=IRSwapStructure.OUTRIGHT),
        IRSwapQuery(tenor="05Yx10Y", structure=IRSwapStructure.OUTRIGHT),
    ],
    n_jobs=-1,
)

PRICING IRSWAPS...: 0it [00:00, ?it/s]


In [5]:
ts_df

,USD-SOFR-1D 1Yx10Y OUTRIGHT RATE,USD-SOFR-1D 05Yx10Y OUTRIGHT RATE,USD-SOFR-1D 10Yx10Y OUTRIGHT RATE
Date,,,
2020-01-02 17:00:00-05:00,1.650152,1.846128,1.927427
2020-01-03 17:00:00-05:00,1.546969,1.738960,1.828376
2020-01-06 17:00:00-05:00,1.539214,1.733291,1.827276
2020-01-07 17:00:00-05:00,1.573113,1.772954,1.864567
2020-01-08 17:00:00-05:00,1.622194,1.822104,1.923579
...,...,...,...
2025-05-05 17:00:00-04:00,3.839351,4.236885,4.348576
2025-05-06 17:00:00-04:00,3.800225,4.213016,4.341714
2025-05-07 17:00:00-04:00,3.763311,4.169355,4.306954
